<a href="https://colab.research.google.com/github/leesolhahaha/Data_Analysis_with_Python/blob/main/11_%5BPandas%5D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A7%91%EA%B3%84_groupby%2C_crosstab%2C_pivot%2C_stack%2C_melt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 그룹바이는 항상 범주형데이터를 기준으로 삼는다 정수나 문자열로 표현된 데이터로서 딱 떨어지는 데이터
# 수치를 보여주고 계산을 보여주는 것을 연속형 데이터라고 하는 데 스룹핑을 할 수 없다 (집계의 기준 불가능)
# 연속형 데이터를 범주형데이터로 바꾼다면 (구간을 만드는 등(도수분포표)) 그룹핑 가능
# 이름은 문자열 데이터긴 하지만 다 다르기 때문에 그룹핑의 기준이 되기 힘들다

In [ ]:
import pandas as pd

df = pd.read_csv("./data/titanic_train.csv", index_col="PassengerId")
df.head()

# groupby

## 단일 그룹
|함수|설명|
|:--|:--|
|count()|행의 갯수|
|nunique()|행의 유니크한 갯수|
|sum()|합|
|mean()|평균|
|min()|최솟값|
|max()|최댓값|
|std()|표준편차|
|var()|분산|

In [ ]:
# 그룹 별로 데이터 모으기.
df.groupby("Pclass") # 1등급, 2등급, 3등급 그룹이 만들어진다

In [ ]:
# 그룹별로 모인 데이터에 대한 계산을 수행한다.
#  그룹 내의 데이터가 N개 였다면, 계산을 수행하면 1개가 된다. N -> 1 (Reduction)
df.groupby("Pclass").count() # 그룹으로 지정한 데이터가 인덱스가 되는 것을 볼 수 있음
# NaN값은 집계에서 빠짐
# count같은 경우는 단지 숫자를 세기만 하는 것이기때문에 name같은 것도 수치 계산이 된다

,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,
1,216,216,216,186,216,216,216,216,176,214
2,184,184,184,173,184,184,184,184,16,184
3,491,491,491,355,491,491,491,491,12,491


In [ ]:
df.groupby("Pclass").nunique() # 유니크한 데이터의 개수

,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,
1,2,216,2,57,4,4,147,94,133,3
2,2,184,2,57,4,4,140,42,7,3
3,2,491,2,68,7,7,394,119,7,3


In [ ]:
df.groupby("Pclass").sum(numeric_only=True) # 문자열을 sum하면 할 수 는 있지만 문자열을 그저 이어붙인 형태가 됨
# numeric_only=True 문자열을 빼기 위해서는 숫자만 계산함

,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,
1,136,7111.42,90,77,18177.4125
2,87,5168.83,74,70,3801.8417
3,119,8924.92,302,193,6714.6951


In [ ]:
# 평균, 최솟값, 최댓값, 분산, 표준편차도 구해보세요!

In [ ]:
# 특정 칼럼에 대해서만 집계하기
df.groupby("Pclass").sum()["Age"] # 별로 좋지 않은 방법
# 다름 컬럼에 대한 집계가 먼저 일어나고 특정 값만 추출되는 것이기 때문에 효율적이지 않다

,Age
Pclass,
1,7111.42
2,5168.83
3,8924.92


In [ ]:
df.groupby("Pclass")["Age"].sum() # 내가 필요한 컬럼만 뽑아서 집계하는 것이 더 효율적인 방법

,Age
Pclass,
1,7111.42
2,5168.83
3,8924.92


In [ ]:
# 나이, 요급 합계
df.groupby("Pclass")[["Age", "Fare"]].sum() # 팬시 인덱싱 사용하면 두개의 합계를 구할 수 있음

,Age,Fare
Pclass,,
1,7111.42,18177.4125
2,5168.83,3801.8417
3,8924.92,6714.6951


## 다중 그룹

In [ ]:
df.groupby(["Sex", "Pclass"]).mean(numeric_only=True) # 리스트 형태로 묶어서 다중 그룹을 만들 수 있음 이것을 투 레벨이라고 함

Survived        Age     SibSp     Parch        Fare
Sex    Pclass                                                     
female 1       0.968085  34.611765  0.553191  0.457447  106.125798
       2       0.921053  28.722973  0.486842  0.605263   21.970121
       3       0.500000  21.750000  0.895833  0.798611   16.118810
male   1       0.368852  41.281386  0.311475  0.278689   67.226127
       2       0.157407  30.740707  0.342593  0.222222   19.741782
       3       0.135447  26.507589  0.498559  0.224784   12.661633

In [ ]:
df.groupby(["Pclass", "Sex"]).mean(numeric_only=True) # 두 기준의 순서를 바꾸면 데이터프레임의 모양이 달라지게 됨 집계에는 영향 없음

Survived        Age     SibSp     Parch        Fare
Pclass Sex                                                        
1      female  0.968085  34.611765  0.553191  0.457447  106.125798
       male    0.368852  41.281386  0.311475  0.278689   67.226127
2      female  0.921053  28.722973  0.486842  0.605263   21.970121
       male    0.157407  30.740707  0.342593  0.222222   19.741782
3      female  0.500000  21.750000  0.895833  0.798611   16.118810
       male    0.135447  26.507589  0.498559  0.224784   12.661633

# crosstab
- 범주형 데이터를 비교분석할 때 유용
- 행에 배치될 데이터( 그룹핑 ), 열에 배치될 데이터( 계산 대상 )을 따로 지정

비율 계산

In [ ]:
pd.crosstab(df["Sex"], df["Survived"]) # 앞쪽은 행(고유값의미) 뒤쪽은 열로 나뉘고 그 안에서 집계가됨 성별에 대한 범주형 타겟
# 기본은 카운트

Survived,0,1
Sex,,
female,81,233
male,468,109


In [ ]:
pd.crosstab(df["Pclass"], df["Survived"])

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


## 범주별 비율 구하기
- `normalize = 'all'`: 전체 합이 100%
- `normalize = 'index'`: 행별 합이 100%
- `normalize = 'columns'`: 열별 합이 100%

여성이 100명이다 라고만 하면 이게 많은 건지 적은건지 절대적인 숫자만 알기때문에 모르는데

여성이 90%다 라고 하면 비율값을 알기 때문에 많다는 것을 알 수 있다.

In [ ]:
pd.crosstab(df["Sex"], df["Survived"], normalize='all')
# 안쪽의 모든 합계를 다 합치면 1이 되는 것을 all방식이라고 함

Survived,0,1
Sex,,
female,0.090909,0.261504
male,0.525253,0.122334


In [ ]:
pd.crosstab(df["Sex"], df["Survived"], normalize='all', margins=True) # margins=True 총 합계까지 볼 수 있음

Survived,0,1,All
Sex,,,
female,0.090909,0.261504,0.352413
male,0.525253,0.122334,0.647587
All,0.616162,0.383838,1.000000


In [ ]:
# index 방식 -> 행 방향의 데이터가 기준이 된다.
#  ex) female 중 사망한 사람이 0.257962, female 중 생존한 사람이 0.742038 ( 행의 합이 1.0 )
#      male 중 사망한 사람이 0.811092, male 중 생존한 사람이 0.188908 (행의 합이 1.0 )
#      전체 중 사망한 사람이 0.616162, 전체 중 생존한 사람이 0.383838 ( 행의 합이 1.0 )
pd.crosstab(df['Sex'], df['Survived'], normalize='index', margins=True)

Survived,0,1
Sex,,
female,0.257962,0.742038
male,0.811092,0.188908
All,0.616162,0.383838


In [ ]:
# columns 방식 -> 열 방향의 데이터가 기준이 된다.
#  ex) 사망한 사람 중 female이 0.147541, male이 0.852459( 열의 합이 1.0 )
#      생존한 사람 중 female이 0.681287, male이 0.318713( 열의 합이 1.0 )
#      전체 인원 중 female이 0.352413, male이 0.647587 ( 열의 합이 1.0 )
pd.crosstab(df['Sex'], df['Survived'], normalize='columns', margins=True)

Survived,0,1,All
Sex,,,
female,0.147541,0.681287,0.352413
male,0.852459,0.318713,0.647587


## 다중 인덱스, 다중 컬럼의 범주표

In [ ]:
pd.crosstab(index=[df['Sex'], df['Pclass']], columns=df['Survived'])

Survived         0   1
Sex    Pclass         
female 1         3  91
       2         6  70
       3        72  72
male   1        77  45
       2        91  17
       3       300  47

In [ ]:
pd.crosstab(index=[df['Sex'], df['Pclass']], columns=df['Survived'], normalize='all')

Survived              0         1
Sex    Pclass                    
female 1       0.003367  0.102132
       2       0.006734  0.078563
       3       0.080808  0.080808
male   1       0.086420  0.050505
       2       0.102132  0.019080
       3       0.336700  0.052750

In [ ]:
pd.crosstab(index=[df['Sex'], df['Pclass']], columns=[df['Survived'], df['Embarked']])

Survived        0            1        
Embarked        C   Q    S   C   Q   S
Sex    Pclass                         
female 1        1   0    2  42   1  46
       2        0   0    6   7   2  61
       3        8   9   55  15  24  33
male   1       25   1   51  17   0  28
       2        8   1   82   2   0  15
       3       33  36  231  10   3  34

# pivot table

pivot이란 수직으로 돌리는 것 (행 방향에 있는 데이터를 열 방향으로 올려주는 것 index를 columns화 시키는 것)

## 단일 인덱스, 단일 컬럼, 단일 값

In [ ]:
pd.pivot_table(
    df,
    index="Sex", # 그룹의 기준을 잡은 것
    columns="Pclass", # 어떠한 데이터를 볼지
    values="Fare", # 값을 계산할 정보
    aggfunc="mean" # 값에 어떠한 집계를 할지
)
# pivot table은 세 가지 정보가 필요함 values 포함

Pclass,1,2,3
Sex,,,
female,106.125798,21.970121,16.118810
male,67.226127,19.741782,12.661633


In [ ]:
pd.pivot_table(
    df,
    index="Sex",
    columns="Pclass",
    values="Fare",
    aggfunc="mean",
    margins=True # 집계의 합을 볼 수 있음
)

Pclass,1,2,3,All
Sex,,,,
female,106.125798,21.970121,16.118810,44.479818
male,67.226127,19.741782,12.661633,25.523893
All,84.154687,20.662183,13.675550,32.204208


## 다중 인덱스, 다중 컬럼, 다중 값

In [ ]:
# 집계를 여러 가지 방식으로 수행할 수 있다.
pd.pivot_table(
    df,
    index='Sex',
    columns='Pclass',
    values='Fare',
    aggfunc=['mean', 'max', 'min'], # 리스트 형식으로
)

mean                             max                   min  \
Pclass           1          2          3         1     2      3        1   
Sex                                                                        
female  106.125798  21.970121  16.118810  512.3292  65.0  69.55  25.9292   
male     67.226127  19.741782  12.661633  512.3292  73.5  69.55   0.0000   

                    
Pclass     2     3  
Sex                 
female  10.5  6.75  
male     0.0  0.00

In [ ]:
pd.pivot_table(
    df,
    index="Sex",
    columns="Pclass",
    values=["Fare", "Age"], # 두 개의 값을 볼 수 도 있음
    aggfunc="mean"
)

Age                              Fare                      
Pclass          1          2          3           1          2          3
Sex                                                                      
female  34.611765  28.722973  21.750000  106.125798  21.970121  16.118810
male    41.281386  30.740707  26.507589   67.226127  19.741782  12.661633

In [ ]:
pd.pivot_table(
    df,
    index=['Sex', 'Pclass'],
    columns='Survived',
    values='Fare',
    aggfunc='mean'
)

Survived                0           1
Sex    Pclass                        
female 1       110.604167  105.978159
       2        18.250000   22.288989
       3        19.773093   12.464526
male   1        62.894910   74.637320
       2        19.488965   21.095100
       3        12.204469   15.579696

In [ ]:
pd.pivot_table(
    df,
    index=['Sex', 'Pclass'],
    columns=['Survived', 'Embarked'],
    values='Fare',
    aggfunc='mean'
)

Survived               0                                  1             \
Embarked               C          Q           S           C          Q   
Sex    Pclass                                                            
female 1       28.712500        NaN  151.550000  117.710019  90.000000   
       2             NaN        NaN   18.250000   25.268457  12.350000   
       3       14.653125  10.904633   21.969018   14.717220  10.084033   
male   1       80.847160  90.000000   53.563316  112.197806        NaN   
       2       25.418225  12.350000   18.997561   25.433350        NaN   
       3        8.720842  11.841550   12.758689   11.435840  12.916667   

Survived                  
Embarked               S  
Sex    Pclass             
female 1       96.743298  
       2       22.272951  
       3       13.171842  
male   1       51.832739  
       2       20.516667  
       3       17.033450

# stack, unstack, melt

- 집계가 불가능한 형식의 데이터를 집계할 때 가능하게 만들어주는 것을 mel
    - 집계의 판별 방법은 groupby가 가능한지 불가능 한지에 따라 결정 됨
    - 컬럼의 카테고리 정보가 컬럼에 존재하는 경우, 컬럼 이름에 컬럼 정보가 포함 되는 순간 불가능
    - 등급 레벨이 row로 들어가 있으면 집계가 가능해짐?
    - select를할 데이터인지 집계를 할 데이터인지를 잘 봐야 함
    
선택할 수 없는 것을 선택 할 수 있게 하는 것 pivot

pivot은 올린 것이었고 melt는 밑으로 내린 것 가로로 있던 것을 열로 만드는 것

## stack, unstack
- stack : 컬럼 레벨에서 인덱스 레벨로 데이터프레임을 변경합니다.
- unstack : 인덱스 레벨에서 컬럼 레벨로 데이터프레임을 변경합니다.
  - stack의 반대입니다.

In [ ]:
pivot = pd.pivot_table(
    df,
    index=['Sex', 'Pclass'],
    values=['Survived', "Fare"],
    aggfunc=['mean', 'median', 'sum']
)
pivot

mean              median                 sum         
                     Fare  Survived      Fare Survived       Fare Survived
Sex    Pclass                                                             
female 1       106.125798  0.968085  82.66455      1.0  9975.8250       91
       2        21.970121  0.921053  22.00000      1.0  1669.7292       70
       3        16.118810  0.500000  12.47500      0.5  2321.1086       72
male   1        67.226127  0.368852  41.26250      0.0  8201.5875       45
       2        19.741782  0.157407  13.00000      0.0  2132.1125       17
       3        12.661633  0.135447   7.92500      0.0  4393.5865       47

In [ ]:
pivot.index # 멀티 행 인덱스 형식으로 나옴 레벨별로 나오는 것

MultiIndex([('female', 1),
            ('female', 2),
            ('female', 3),
            (  'male', 1),
            (  'male', 2),
            (  'male', 3)],
           names=['Sex', 'Pclass'])

In [ ]:
pivot.columns # 멀티 칼럼 형식으로 나옴 레벨별로 나오는 것

MultiIndex([(  'mean',     'Fare'),
            (  'mean', 'Survived'),
            ('median',     'Fare'),
            ('median', 'Survived'),
            (   'sum',     'Fare'),
            (   'sum', 'Survived')],
           )

In [ ]:
# stack
pivot.stack(0) # 컬럼의 첫번째 레벨을 인덱스로 내린다
# 컬럼에 있던 정보가 행인덱스로 가게 됨

<ipython-input-38-ce7accd6e29f>:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  pivot.stack(0) # 컬럼의 첫번째 레벨을 인덱스로 내린다.


Fare   Survived
Sex    Pclass                               
female 1      mean     106.125798   0.968085
              median    82.664550   1.000000
              sum     9975.825000  91.000000
       2      mean      21.970121   0.921053
              median    22.000000   1.000000
              sum     1669.729200  70.000000
       3      mean      16.118810   0.500000
              median    12.475000   0.500000
              sum     2321.108600  72.000000
male   1      mean      67.226127   0.368852
              median    41.262500   0.000000
              sum     8201.587500  45.000000
       2      mean      19.741782   0.157407
              median    13.000000   0.000000
              sum     2132.112500  17.000000
       3      mean      12.661633   0.135447
              median     7.925000   0.000000
              sum     4393.586500  47.000000

In [ ]:
pivot.stack(1) # 컬럼의 두 번째 레벨을 인덱스로 내린다.

<ipython-input-39-30ec8451bfeb>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  pivot.stack(1) # 컬럼의 두 번째 레벨을 인덱스로 내린다.


mean    median        sum
Sex    Pclass                                          
female 1      Fare      106.125798  82.66455  9975.8250
              Survived    0.968085   1.00000    91.0000
       2      Fare       21.970121  22.00000  1669.7292
              Survived    0.921053   1.00000    70.0000
       3      Fare       16.118810  12.47500  2321.1086
              Survived    0.500000   0.50000    72.0000
male   1      Fare       67.226127  41.26250  8201.5875
              Survived    0.368852   0.00000    45.0000
       2      Fare       19.741782  13.00000  2132.1125
              Survived    0.157407   0.00000    17.0000
       3      Fare       12.661633   7.92500  4393.5865
              Survived    0.135447   0.00000    47.0000

In [ ]:
pivot.unstack(0) # 행 인덱스의 첫 번째 레벨을 컬럼으로 올린다.

mean                                   median                    \
              Fare             Survived                Fare          Survived   
Sex         female       male    female      male    female     male   female   
Pclass                                                                          
1       106.125798  67.226127  0.968085  0.368852  82.66455  41.2625      1.0   
2        21.970121  19.741782  0.921053  0.157407  22.00000  13.0000      1.0   
3        16.118810  12.661633  0.500000  0.135447  12.47500   7.9250      0.5   

                   sum                           
                  Fare            Survived       
Sex    male     female       male   female male  
Pclass                                           
1       0.0  9975.8250  8201.5875       91   45  
2       0.0  1669.7292  2132.1125       70   17  
3       0.0  2321.1086  4393.5865       72   47

In [ ]:
pivot.unstack(1) # 행 인덱스의 두번째 레벨을 컬럼으로 올린다

mean                                                      \
              Fare                        Survived                       
Pclass           1          2          3         1         2         3   
Sex                                                                      
female  106.125798  21.970121  16.118810  0.968085  0.921053  0.500000   
male     67.226127  19.741782  12.661633  0.368852  0.157407  0.135447   

          median                                         sum             \
            Fare               Survived                 Fare              
Pclass         1     2       3        1    2    3          1          2   
Sex                                                                       
female  82.66455  22.0  12.475      1.0  1.0  0.5  9975.8250  1669.7292   
male    41.26250  13.0   7.925      0.0  0.0  0.0  8201.5875  2132.1125   

                                    
                  Survived          
Pclass          3        1   2   3  
Sex                                 
female  2321.1086       91  70  72  
male    4393.5865       45  17  47

## melt
- `pd.melt(데이터명, id_vars=기준 컬럼)`



![이미지](https://i0.wp.com/cmdlinetips.com/wp-content/uploads/2019/06/Pandas_melt_reshape.png?w=421&ssl=1)

보기에 좋은 데이터지 분석이나 집계에 좋은 데이터는 아님
그걸 집계하기 좋은 데이터로 바꿔주는 거이 melt, 데이터의 개수만큼 행이 늘어남

In [ ]:
data = pd.DataFrame({'name':['a','b','c']
                    , 'order_count':[3,4,10]
                    , 'amount':[10000,25000,300000]})
data

,name,order_count,amount
0,a,3,10000
1,b,4,25000
2,c,10,300000


In [ ]:
pd.melt(
    data,
    id_vars=["name"] # 인덱스처럼 사용할 데이터 (인덱슨느 아님) 기준이 되는 데이터, 중복가능
)
# 컬럼의 이름으로 사용되는 데이터들이 값으로 들어가는 것
# 컬럼의 정보가 행단위로 들어가게 됨(레코드화)

,name,variable,value
0,a,order_count,3
1,b,order_count,4
2,c,order_count,10
3,a,amount,10000
4,b,amount,25000
5,c,amount,300000


In [ ]:
pd.melt(
    data,
    id_vars=["name"],
    var_name="type",
    value_name="val"
)

,name,type,val
0,a,order_count,3
1,b,order_count,4
2,c,order_count,10
3,a,amount,10000
4,b,amount,25000
5,c,amount,300000
